## Schema of our building
Inserer image

In [ ]:
[!bldg](builidng.pdf)

## Dimensions of the building

Here, we compute the dimensions of our 2 rooms:
we have chosen to take into account only one wall in contact with the rest of the chalet and one wall with the 2m² window in contact with the outside

In [ ]:
l_br=3 #lenght of the bedroom
L_br=4 #width of the bedroom
l_d= 3 #lenght of the dressing
L_d= 2 #width of the dressing
height= 3   #height under ceiling

S_br= l_br*L_br  #total surface bedroom
S_d= l_d*L_d    #total surface dressing

S_glass= 2   # surface of the glass taken into account in the study

S_wall_3=l_d*height  # surface of the wall between the dressing and the bedroom
S_wall_4=L_d*height  # surface of the wall of the dressing
S_wall_2= L_br*height # surface de droite 
S_wall_1= l_br*height - S_glass #surface du haut
 



h_in = 8 #convection coefficient for inside, W/m² K;
h_out= 25 #convection coefficient for outside, W/m² K;
Tin = 20    #temperature inside, K;
σ = σ = 5.67e-8     # W/(m²⋅K⁴) Stefan-Bolzmann constant


## Desciption of the building
inserer photo adri

## Thermo-physical properties

In [ ]:
wood = {'Conductivity': 0.06,
            'Density': ,
            'Specific heat': ,
            'Width':0.2 ,
            'Surface': S_wall_1+S_wall_2+S_wall_3+S_wall_4
             }

whool = {'Conductivity': 0.034,
              'Density': ,
              'Specific heat': ,
              'Width':0.25 ,
              'Surface': S_wall_1+S_wall_2+S_wall_3+S_wall_4 }

glass = {'Conductivity': 1 ,
         'Density': ,
         'Specific heat': ,
         'Width':0.04 ,
         'Surface': S_glass}

air = {'Conductivity': 0.024 ,
         'Density': ,
         'Specific heat': ,
         'Width':0.1 ,
         'Surface': S_glass }   
          

## Radiative properties

In [ ]:
ε_wLW = 0.85    # long wave emmisivity: wall surface (concrete)
ε_gLW = 0.90    # long wave emmisivity: glass pyrex
α_wSW = 0.25    # short wave absortivity: white smooth surface
α_gSW = 0.38    # short wave absortivity: reflective blue glass
τ_gSW = 0.30    # short wave transmitance: reflective blue glass

## Thermal circuit
Inserer photo


## Thermal conductances

### Conduction

In [ ]:
G_1 = wood['Conductivity'] / wood['Width'] * S_wall_1
G_2 = wood['Conductivity'] / wood['Width'] * S_wall_1
G_3 = whool['Conductivity'] / whool['Width'] * S_wall_1
G_4 = whool['Conductivity'] / whool['Width'] * S_wall_1
G_10= wood['Conductivity'] / wood['Width'] * S_wall_2
G_17= wood['Conductivity'] / wood['Width'] * S_wall_3
G_14= wood['Conductivity'] / wood['Width'] * S_wall_4
G_13= glass['Conductivity'] / glass['Width'] * S_glass
G_12= glass['Conductivity'] / glass['Width'] * S_glass

### Convection

In [ ]:
G_0 = h_out * S_wall_1
G_5= h_in * S_wall_1
G_11= h_in * S_wall_2
G_16= h_in * S_wall_3
G_15= h_in * S_wall_4


### Long wave radiation

In [ ]:
#Long Wave radiation
Tm = Tin + 273   # K, mean temp for radiative exchange
Fwg = glass['Surface']  / wood['Surface']    # view factor wall-glass

#Between window and wall_3
G7_1 = 4 * σ * Tm**3 * ε_wLW / (1 - ε_wLW) * S_wall_3
G7_12 = 4 * σ * Tm**3 * Fwg * S_wall_3 
G7_2 = 4 * σ * Tm**3 * ε_gLW / (1 - ε_gLW) * glass['Surface']
G7 = 1 / (1 / G7_1 + 1 / G7_12 + 1 / G7_2) 

#Between wall_1 and wall_3
G8_1 = 4 * σ * Tm**3 * ε_wLW / (1 - ε_wLW) * S_wall_3
G8_12 = 4 * σ * Tm**3 * Fwg * S_wall_3 
G8_2 = 4 * σ * Tm**3 * ε_gLW / (1 - ε_gLW) * S_wall_1 
G8 = 1 / (1 / G8_1 + 1 / G8_12 + 1 / G8_2)

## Thermal capacities

In [ ]:
C1 = wood['Density'] * wood['Specific heat'] * wood['Surface'] * wood['Width']      #wood capacity wall 1
C2 = whool['Density'] * whool['Specific heat'] * whool['Surface'] * whool['Width']  #whool capacity wall 1

## System of Algebraic Differential Equations (DAE)

### Incidence matrix A

Photo Matrice A

In [ ]:
import numpy as np
nq = 18                         # n° of branches
n0 = 13                         # n° of nodes
A = np.zeros(nq,n0)             # n° of branches X n° of nodes
A[0, 0] = 1                     # branch 0: -> node 0
A[1, 0], A[1, 1] = -1, 1        # branch 1: node 0 -> node 1
A[2, 1], A[2, 2] = -1, 1        # branch 2: node 1 -> node 2
A[3, 2], A[3, 3] = -1, 1        # branch 3: node 2 -> node 3
A[4, 3], A[4, 4] = -1, 1        # branch 4: node 3 -> node 4
A[5, 4], A[5, 5] = -1, 1        # branch 5: node 4 -> node 5
A[6, 5], A[6, 6] = -1, 1        # branch 6: node 5 -> node 6
A[7, 6], A[7, 7] = -1, 1        # branch 7: node 6 -> node 7
A[8, 4], A[8, 6] = -1, 1        # branch 8: node 4 -> node 6
A[9, 5], A[9, 7] =  1,-1        # branch 9: node 7 -> node 5
A[10, 8] = 1                    # branch 10: -> node 8
A[11, 5], A[11, 8] = 1, -1      # branch 11: node 8 -> node 5
A[12, 9] = 1                    # branch 12: -> node 9
A[13, 7], A[13, 9] = 1, -1      # branch 13: node 9 -> node 7
A[14, 12] = 1                   # branch 14: -> node 12
A[15, 11], A[15, 12] = 1, -1    # branch 15: node 12 -> node 11  
A[16, 10], A[16, 11] = 1, -1    # branch 16: node 11 -> node 10
A[17, 6], A[17, 10] = 1, -1     # branch 17: node 10 -> node 6

### Conductance matrix G

Photo matrice G

In [ ]:
G = np.diag(G0,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17)

### Capacity Matrix C

Photo matrice C

In [ ]:
C = np.diag(0,C1,0,C3,0,0,0,0,0,0,0,0,0)

### Temperature vector Teta

In [ ]:
Teta = np.zeros(n0,1)

### Temperature source vector b

In [ ]:
b = np.zeros(nq,1)
b[0] = Ti0
b[8] = Tin
b[9] = Ti0
b[12] = Tin

### Heat flow source vector f

In [ ]:
f = np.zeros(n0,1)
f[0] = Phi0
f[9] = Phia

### Heat flow vector: q

In [ ]:
q = np.zeros(nq,1)